This notebook allows collecting results across multiple pipeline runs. Concatenating across sessions and bin sizes. 
It takes CSVs, then determines the most recent one from the filename. 

There are two major sets of decoded epochs - Laps and Ripples/Replays
There are two sets of marginals for the decoded epochs - the "by epoch" and "by time bin" marginals.
The "by time bin" epochs are a larger granulation, with each epoch consisting of one or more time bin.


```python

# 2024-01-23 - 
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-fet11-01_12-58-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-03_12-3-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_17-46-44_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-11-02_19-28-0_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-58-3_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-10_12-25-50_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_16-40-54_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_22-24-40_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_16-53-46_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-4-09_17-29-30_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_14-26-15_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-09_1-22-43_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-07_16-40-19_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-08_21-16-25_time_bin_size_sweep_results.h5
# C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-23_GL-2006-6-12_15-55-31_time_bin_size_sweep_results.h5



# found_session_export_paths = [Path(v).resolve() for v in  ["C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0420PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0645PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(laps_marginals_df).csv",
# "C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs/2024-01-12_0828PM-kdiba_pin01_one_fet11-01_12-58-54-(ripple_marginals_df).csv",
# ]]

[
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-17_0210AM-kdiba_vvp01_one_2006-4-09_17-29-30-(laps_simple_pf_pearson_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(laps_weighted_corr_merged_df)_tbin-0.025.csv",
    "2024-02-19_0430PM-kdiba_vvp01_one_2006-4-10_12-25-50-(ripple_weighted_corr_merged_df)_tbin-0.025.csv",
]

```

In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib auto

# Switch to the desired interactivity mode
plt.interactive(True)

# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard
from pyphocorehelpers.Filesystem.path_helpers import file_uri_from_path, sanitize_filename_for_Windows
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files, read_and_process_csv_file
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_histograms, plot_stacked_histograms

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_files, _process_and_load_exported_file, _common_cleanup_operations, convert_to_dataframe

from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import recover_user_annotation_and_is_valid_columns

debug_print: bool = False
enable_neptune: bool = False

# TODAY_DAY_DATE: str = f"2024-06-12_Apogee"
# TODAY_DAY_DATE: str = f"2024-06-12_GL"
TODAY_DAY_DATE: str = f"2024-06-12_rMBP"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

    ## Gets the notebook filepath for Neptune:
    import IPython
    from pyphocorehelpers.programming_helpers import IPythonHelpers
    notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
    assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"
    # notebook_filepath

    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortAcrossSessions",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
            
    neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])


    if neptuner.run is None:
        neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])
        params = {"TODAY_DAY_DATE": TODAY_DAY_DATE, "run_workstation": "Apogee"}
        neptuner.run["parameters"] = params
        neptuner.outputs = neptuner.run['outputs']
        neptuner.figures = neptuner.outputs['figures']

    neptuner_run: AutoValueConvertingNeptuneRun = neptuner.run
    
    # run = neptune.init_run(source_files=["**/*.dvc"])

    # # Pre-execution dataframe view:
    # run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"

else:
    # no neptune:
    neptuner = None    
    neptuner_run = None

# if enable_neptune:
#     run["dataset/latest"].track_files(f"file://{global_batch_result_file_path}") # "s3://datasets/images" # update file progress post-load
#     # Post-execution dataframe view:
#     run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"
        
# if enable_neptune:
#     ## POST Run
#     project.stop()

# from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables
# run = neptune.init_run(project='commander.pho/PhoDibaLongShortAcrossSessions')
# import neptune

# run = neptune.init_run(
#     project="commander.pho/PhoDibaLongShortAcrossSessions",
#     api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ==",
# )  # your credentials

# params = {"learning_rate": 0.001, "optimizer": "Adam"}
# run["parameters"] = params

# for epoch in range(10):
#     run["train/loss"].append(0.9 ** epoch)

# run["eval/f1_score"] = 0.66

# run.stop()

known_bad_sessions = [IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')]
known_bad_session_strs = [str(v.get_description()) for v in known_bad_sessions]
known_bad_session_strs

Automatic pdb calling has been turned OFF
TODAY_DAY_DATE: 2024-06-12_rMBP


['kdiba_pin01_one_11-02_17-46-44']

# Definitions
"_new" (as in `all_sessions_new_ripple_df`) adds simple_pearson and a few other measures

### csv_files

In [2]:
## Load across session t_delta CSV, which contains the t_delta for each session:

## INPUTS: known_bad_session_strs,

cuttoff_date = datetime(2024, 5, 18)
# cuttoff_date = None

# t_delta_csv_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\output\collected_outputs\2024-01-18_GL_t_split_df.csv').resolve() # Apogee
# t_delta_csv_path = Path('/home/halechr/cloud/turbo/Data/Output/collected_outputs/2024-01-18_GL_t_split_df.csv').resolve() # GL

# collected_outputs_directory = '/home/halechr/FastData/collected_outputs/'
# collected_outputs_directory = r'C:\Users\pho\Desktop\collected_outputs'
# collected_outputs_directory = r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs' # APOGEE
# collected_outputs_directory = '/home/halechr/cloud/turbo/Data/Output/collected_outputs' # GL

known_collected_outputs_paths = [Path(v).resolve() for v in ['/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs',
                                                            '/Volumes/SwapSSD/Data/collected_outputs', r"K:/scratch/collected_outputs", '/Users/pho/Dropbox (University of Michigan)/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs',
                                                            '/home/halechr/FastData/collected_outputs/', '/home/halechr/cloud/turbo/Data/Output/collected_outputs']]
collected_outputs_directory = find_first_extant_path(known_collected_outputs_paths)
assert collected_outputs_directory.exists(), f"collected_outputs_directory: {collected_outputs_directory} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_directory: {collected_outputs_directory}')

# Create a 'figures' subfolder if it doesn't exist
figures_folder: Path = collected_outputs_directory.joinpath('figures').resolve()
figures_folder.mkdir(parents=False, exist_ok=True)
assert figures_folder.exists()
print(f'\tfigures_folder: {file_uri_from_path(figures_folder)}')

# Create an output path for the across session collected results (like the aggregate CSVs built from the individual session CSVs)
across_sessions_output_folder: Path = collected_outputs_directory.joinpath('../across_sessions').resolve()
across_sessions_output_folder.mkdir(parents=False, exist_ok=True)
assert across_sessions_output_folder.exists()
print(f'\tacross_sessions_output_folder: {file_uri_from_path(across_sessions_output_folder)}')

# t_delta_csv_path = collected_outputs_directory.joinpath('2024-01-18_GL_t_split_df.csv').resolve() # GL
t_delta_csv_path = collected_outputs_directory.joinpath('../2024-01-18_GL_t_split_df.csv').resolve()
assert t_delta_csv_path.exists(), f"t_split_df CSV at '{t_delta_csv_path}' does not exist!"

if neptuner_run is not None:
    _neptuner_run_parameters = dict(cuttoff_date=cuttoff_date, collected_outputs_directory=collected_outputs_directory.as_posix(), figures_folder=figures_folder.as_posix(),
                           across_sessions_output_folder=across_sessions_output_folder.as_posix(), t_delta_csv_path=t_delta_csv_path.as_posix())
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parameters/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

## Find the files:
csv_files = find_csv_files(collected_outputs_directory)
h5_files = find_HDF5_files(collected_outputs_directory)

csv_sessions = find_most_recent_files(found_session_export_paths=csv_files, cuttoff_date=cuttoff_date)
h5_sessions = find_most_recent_files(found_session_export_paths=h5_files)


_neptuner_run_parameters = dict(csv_files=csv_files, h5_files=h5_files,
                            csv_sessions=csv_sessions, h5_sessions=h5_sessions)

## The CSV containing the session delta time:
t_delta_df = pd.read_csv(t_delta_csv_path, index_col=0) # Assuming that your CSV file has an index column
# adds `delta_aligned_t_start`, `delta_aligned_t_end` columns
t_delta_df['delta_aligned_t_start'] = t_delta_df['t_start'] - t_delta_df['t_delta']
t_delta_df['delta_aligned_t_end'] = t_delta_df['t_end'] - t_delta_df['t_delta']

# computes `earliest_delta_aligned_t_start`, latest_delta_aligned_t_end
earliest_delta_aligned_t_start: float = np.nanmin(t_delta_df['delta_aligned_t_start'])
latest_delta_aligned_t_end: float = np.nanmax(t_delta_df['delta_aligned_t_end'])
print(f'earliest_delta_aligned_t_start: {earliest_delta_aligned_t_start}, latest_delta_aligned_t_end: {latest_delta_aligned_t_end}')
t_delta_dict = t_delta_df.to_dict(orient='index')
# t_delta_df

# #TODO 2024-03-02 12:12: - [ ] Could add weighted correlation if there is a dataframe for that and it's computed:
_df_raw_variable_names = ['simple_pf_pearson_merged_df', 'weighted_corr_merged_df']
_df_variables_names = ['laps_weighted_corr_merged_df', 'ripple_weighted_corr_merged_df', 'laps_simple_pf_pearson_merged_df', 'ripple_simple_pf_pearson_merged_df']

# # tbin_values_dict = {'laps': self.laps_decoding_time_bin_size, 'ripple': self.ripple_decoding_time_bin_size}
time_col_name_dict = {'laps': 'lap_start_t', 'ripple': 'ripple_start_t'} ## default should be 't_bin_center'

# fold older files:
# {'laps_marginals_df': 'lap_start_t', 'ripple_marginals_df': 'ripple_start_t', 'laps_time_bin_marginals_df':'t_bin_center', 'ripple_time_bin_marginals_df':'t_bin_center'}
    
if neptuner_run is not None:
    _neptuner_run_parameters = _neptuner_run_parameters | dict(earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
                                     t_delta_df=t_delta_df)
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing


# csv_sessions
# Extract each of the separate files from the sessions:

# final_sessions: Dict[types.session_str, Dict[str, Path]] = {}

final_sessions_loaded_laps_dict = {}
final_sessions_loaded_ripple_dict = {}
final_sessions_loaded_laps_time_bin_dict = {}
final_sessions_loaded_ripple_time_bin_dict = {}

final_sessions_loaded_simple_pearson_laps_dict = {}
final_sessions_loaded_simple_pearson_ripple_dict = {}

# stupid reudndant but compatible method
final_sessions_loaded_laps_wcorr_dict = {}
final_sessions_loaded_ripple_wcorr_dict = {}

final_sessions_loaded_laps_all_scores_dict = {}
final_sessions_loaded_ripple_all_scores_dict = {}

if cuttoff_date is not None:
    final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (a_path, an_decoding_time_bin_size_str, an_export_datetime) in session_dict.items() if ((an_export_datetime >= cuttoff_date) and (session_str not in known_bad_session_strs))}
                                                                                              for session_str, session_dict in csv_sessions.items() }
else:
    # no cutoff recency date:
    final_sessions: Dict[types.session_str, Dict[str, Path]] = {session_str:{file_type:a_path for file_type, (a_path, an_decoding_time_bin_size_str, an_export_datetime) in session_dict.items() if (session_str not in known_bad_session_strs)}
                                                                                            for session_str, session_dict in csv_sessions.items()}


for session_str, session_dict in final_sessions.items():
    session_name = str(session_str)  # Extract session name from the filename
    if debug_print:
        print(f'processing session_name: {session_name}')
    curr_session_t_delta = t_delta_dict.get(session_name, {}).get('t_delta', None)
    if curr_session_t_delta is None:
        print(f'WARN: curr_session_t_delta is None for session_str = "{session_str}"')

    # Process each file type with its corresponding details
    _process_and_load_exported_file(session_dict, 'laps_marginals_df', final_sessions_loaded_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_marginals_df', final_sessions_loaded_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    _process_and_load_exported_file(session_dict, 'laps_time_bin_marginals_df', final_sessions_loaded_laps_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    _process_and_load_exported_file(session_dict, 'ripple_time_bin_marginals_df', final_sessions_loaded_ripple_time_bin_dict, session_str, curr_session_t_delta, 't_bin_center')
    _process_and_load_exported_file(session_dict, 'laps_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_laps_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_simple_pf_pearson_merged_df', final_sessions_loaded_simple_pearson_ripple_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    _process_and_load_exported_file(session_dict, 'laps_weighted_corr_merged_df', final_sessions_loaded_laps_wcorr_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_weighted_corr_merged_df', final_sessions_loaded_ripple_wcorr_dict, session_str, curr_session_t_delta, 'ripple_start_t')
    
    # _process_and_load_exported_file(session_dict, 'laps_all_scores_merged_df', final_sessions_loaded_laps_all_scores_dict, session_str, curr_session_t_delta, 'lap_start_t')
    _process_and_load_exported_file(session_dict, 'ripple_all_scores_merged_df', final_sessions_loaded_ripple_all_scores_dict, session_str, curr_session_t_delta, 'ripple_start_t')

## Build across_sessions join dataframes:
all_sessions_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_dict.values()), axis='index', ignore_index=True)
# Add 'epoch_idx' column for compatibility:
if all_sessions_laps_df is not None:
    all_sessions_laps_df['epoch_idx'] = all_sessions_laps_df['lap_idx']
if all_sessions_ripple_df is not None:
    all_sessions_ripple_df['epoch_idx'] = all_sessions_ripple_df['ripple_idx']

# *_time_bin marginals:
all_sessions_laps_time_bin_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_time_bin_dict.values()), axis='index', ignore_index=True)
all_sessions_ripple_time_bin_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_time_bin_dict.values()), axis='index', ignore_index=True)

# NEW ________________________________________________________________________________________________________________ #
all_sessions_simple_pearson_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_simple_pearson_laps_dict.values()), axis='index', ignore_index=True)
all_sessions_simple_pearson_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_simple_pearson_ripple_dict.values()), axis='index', ignore_index=True)

if len(final_sessions_loaded_laps_wcorr_dict) > 0:
    all_sessions_wcorr_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_wcorr_dict.values()), axis='index', ignore_index=True)
else:
    all_sessions_wcorr_laps_df = None # empty df would be better

all_sessions_wcorr_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_wcorr_dict.values()), axis='index', ignore_index=True)

# `*_all_scores_*`: __________________________________________________________________________________________________ #
# all_sessions_all_score_laps_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_laps_all_scores_dict.values()), axis='index', ignore_index=True)
all_sessions_all_scores_ripple_df: pd.DataFrame = PandasHelpers.safe_concat(list(final_sessions_loaded_ripple_all_scores_dict.values()), axis='index', ignore_index=True)


# dfs_list = (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)
dfs_dict = dict(zip(('all_sessions_laps_df', 'all_sessions_ripple_df', 'all_sessions_laps_time_bin_df', 'all_sessions_ripple_time_bin_df'), (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)))

# for a_df in dfs_list:
for a_df_name, a_df in dfs_dict.items():
    if a_df is not None:
        if 'time_bin_size' not in a_df:
            print(f'Uh-oh! time_bin_size is missing for "{a_df_name}"! This must be old exports!')
            print(f'\tTry to determine the time_bin_size from the filenames: {csv_sessions}')
            ## manual correction UwU
            time_bin_size: float = 0.025
            print(f'\tWARNING! MANUAL OVERRIDE TIME BIN SIZE SET: time_bin_size = {time_bin_size}. Assigning to dataframes....')
            a_df['time_bin_size'] = time_bin_size
        else:
            # Filter rows based on column: 'time_bin_size'
            a_df = a_df[a_df['time_bin_size'].notna()]


# if 'time_bin_size' not in all_sessions_laps_df:
#     print('Uh-oh! time_bin_size is missing! This must be old exports!')
#     print(f'\tTry to determine the time_bin_size from the filenames: {csv_sessions}')
#     ## manual correction UwU
#     time_bin_size: float = 0.025
#     print(f'WARNING! MANUAL OVERRIDE TIME BIN SIZE SET: time_bin_size = {time_bin_size}. Assigning to dataframes....')
#     all_sessions_laps_df['time_bin_size'] = time_bin_size
#     all_sessions_ripple_df['time_bin_size'] = time_bin_size
#     all_sessions_laps_time_bin_df['time_bin_size'] = time_bin_size
#     all_sessions_ripple_time_bin_df['time_bin_size'] = time_bin_size
#     print(f'\tdone.')
# else:
#     # Filter rows based on column: 'time_bin_size'
#     all_sessions_laps_df = all_sessions_laps_df[all_sessions_laps_df['time_bin_size'].notna()]
#     all_sessions_ripple_df = all_sessions_ripple_df[all_sessions_ripple_df['time_bin_size'].notna()]
#     all_sessions_laps_time_bin_df = all_sessions_laps_time_bin_df[all_sessions_laps_time_bin_df['time_bin_size'].notna()]
#     all_sessions_ripple_time_bin_df = all_sessions_ripple_time_bin_df[all_sessions_ripple_time_bin_df['time_bin_size'].notna()]

all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df)]
all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df)]

# all_sessions_all_score_laps_df, all_sessions_all_scores_ripple_df = [_common_cleanup_operations(a_df) for a_df in (all_sessions_all_score_laps_df, all_sessions_all_scores_ripple_df)]
# all_sessions_all_score_laps_df = _common_cleanup_operations(all_sessions_all_score_laps_df)
all_sessions_all_scores_ripple_df = _common_cleanup_operations(all_sessions_all_scores_ripple_df)

all_sessions_simple_pearson_laps_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_simple_pearson_laps_df, all_sessions_wcorr_laps_df, should_drop_directional_columns=False, start_t_idx_name='delta_aligned_start_t')
all_sessions_simple_pearson_ripple_df: pd.DataFrame = DecoderDecodedEpochsResult.merge_decoded_epochs_result_dfs(all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_ripple_df, should_drop_directional_columns=False, start_t_idx_name='ripple_start_t')

# all_sessions_laps_time_bin_df # 601845 rows × 9 column


## epoch-based ones:
# an_active_df = all_sessions_ripple_df
# all_sessions_all_scores_df = all_sessions_all_scores_ripple_df
# a_time_column_names = 'ripple_start_t'
all_sessions_ripple_df = recover_user_annotation_and_is_valid_columns(all_sessions_ripple_df, all_sessions_all_scores_df=all_sessions_all_scores_ripple_df, a_time_column_names='ripple_start_t')
# all_sessions_ripple_df

# all_sessions_laps_df = recover_user_annotation_and_is_valid_columns(all_sessions_laps_df, all_sessions_all_scores_df=all_sessions_all_score_laps_df, a_time_column_names='delta_aligned_start_t') # needs all_sessions_all_score_laps_df

# ## can't do the time_bin ones yet because it doesn't have 'ripple_start_t' to match on:
# an_active_df = all_sessions_ripple_time_bin_df
# a_time_column_names = 'delta_aligned_start_t'
# all_sessions_all_scores_df = all_sessions_all_scores_ripple_df
# all_sessions_ripple_time_bin_df = recover_user_annotation_and_is_valid_columns(all_sessions_ripple_time_bin_df, all_sessions_all_scores_df=all_sessions_all_scores_df, a_time_column_names='delta_aligned_start_t')

parsed_csv_files_df: pd.DataFrame = convert_to_dataframe(csv_sessions)
parsed_csv_files_df

if neptuner_run is not None:
    _neptuner_run_parameters = dict(parsed_csv_files_df=parsed_csv_files_df,all_sessions_laps_df=all_sessions_laps_df, all_sessions_ripple_df=all_sessions_ripple_df,
                                    all_sessions_laps_time_bin_df=all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df=all_sessions_ripple_time_bin_df,
                                    all_sessions_all_scores_ripple_df=all_sessions_all_scores_ripple_df,
                                    all_sessions_simple_pearson_laps_df=all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df=all_sessions_simple_pearson_ripple_df,
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing


# if (neptuner is not None) and (neptuner_run is not None):
#     neptuner.stop()

collected_outputs_directory: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs
	figures_folder: file:///Users/pho/University%20of%20Michigan%20Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/figures
	across_sessions_output_folder: file:///Users/pho/University%20of%20Michigan%20Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions
ERR: Could not parse filename: "session_h5_files_copydict_2024-04-12_GL"
ERR: Could not parse filename: "session_h5_files_copydict_2024-06-12_Apogee"
ERR: Could not parse filename: "session_h5_files_copydict_2024-06-11_Apogee"
earliest_delta_aligned_t_start: -2057.2259484970764, latest_delta_aligned_t_end: 1661.8560019930592
WARN: curr_session_t_delta is None for session_str = "fet11-01_12-58-54"
session "fet11-01_12-58-54", df_file_name_key: "laps_marginals_df" - did not fully work. (error "'laps_marginals_df'". Skipping.
session "fet11-01_12-58-54", df_file_name_key: "ripp

In [ ]:
neptuner_run.stop()

In [9]:
# pd.set_option('display.min_rows', 50)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.show_dimensions', True)

with pd.option_context('display.min_rows', 50, 'display.show_dimensions', True):
    # max_elements
    display(parsed_csv_files_df)

# parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']


,session,file_type,path,decoding_time_bin_size_str,export_datetime
0,kdiba_gor01_one_2006-6-09_1-22-43,ripple_weighted_corr_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.025,2024-06-12 13:20:00
1,kdiba_gor01_one_2006-6-09_1-22-43,ripple_simple_pf_pearson_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.025,2024-06-12 13:20:00
2,kdiba_gor01_one_2006-6-09_1-22-43,ripple_all_scores_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.025,2024-06-12 13:20:00
3,kdiba_gor01_one_2006-6-09_1-22-43,laps_weighted_corr_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.2,2024-06-12 13:20:00
4,kdiba_gor01_one_2006-6-09_1-22-43,laps_simple_pf_pearson_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.2,2024-06-12 13:20:00
5,kdiba_gor01_one_2006-6-09_1-22-43,ripple_WCorrShuffle_df,/Users/pho/University of Michigan Dropbox/MED-...,0.025,2024-06-04 11:00:00
6,kdiba_gor01_one_2006-6-09_1-22-43,ripple_time_bin_marginals_df,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-05-23 00:00:00
7,kdiba_gor01_one_2006-6-09_1-22-43,ripple_marginals_df,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-05-23 00:00:00
8,kdiba_gor01_one_2006-6-09_1-22-43,laps_time_bin_marginals_df,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-05-23 00:00:00
9,kdiba_gor01_one_2006-6-09_1-22-43,laps_marginals_df,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-05-23 00:00:00


In [ ]:
parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']

all_sessions_all_scores_ripple_df


In [ ]:
all_sessions_ripple_time_bin_df

In [ ]:
print(list(np.unique(parsed_csv_files_df.file_type))) # ['laps_marginals_df', 'laps_simple_pf_pearson_merged_df', 'laps_time_bin_marginals_df', 'laps_weighted_corr_merged_df', 'merged_complete_epoch_stats_df', 'ripple_all_scores_merged_df', 'ripple_marginals_df', 'ripple_simple_pf_pearson_merged_df', 'ripple_time_bin_marginals_df', 'ripple_weighted_corr_merged_df']

In [ ]:
csv_sessions

In [3]:
display(parsed_csv_files_df)

across_sessions_parsed_csv_files_path = across_sessions_output_folder.joinpath(f'{TODAY_DAY_DATE}_parsed_csv_files_df.csv').resolve()
# parsed_csv_files_df.to_clipboard(excel=True)
parsed_csv_files_df.to_csv(across_sessions_parsed_csv_files_path)
display(fullwidth_path_widget(across_sessions_parsed_csv_files_path, file_name_label='across_sessions_parsed_csv_files_path:'))

,session,file_type,path,decoding_time_bin_size_str,export_datetime
0,kdiba_gor01_one_2006-6-09_1-22-43,ripple_weighted_corr_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.025,2024-06-12 13:20:00
1,kdiba_gor01_one_2006-6-09_1-22-43,ripple_simple_pf_pearson_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.025,2024-06-12 13:20:00
2,kdiba_gor01_one_2006-6-09_1-22-43,ripple_all_scores_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.025,2024-06-12 13:20:00
3,kdiba_gor01_one_2006-6-09_1-22-43,laps_weighted_corr_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.2,2024-06-12 13:20:00
4,kdiba_gor01_one_2006-6-09_1-22-43,laps_simple_pf_pearson_merged_df,/Users/pho/University of Michigan Dropbox/MED-...,0.2,2024-06-12 13:20:00
5,kdiba_gor01_one_2006-6-09_1-22-43,ripple_WCorrShuffle_df,/Users/pho/University of Michigan Dropbox/MED-...,0.025,2024-06-04 11:00:00
6,kdiba_gor01_one_2006-6-09_1-22-43,ripple_time_bin_marginals_df,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-05-23 00:00:00
7,kdiba_gor01_one_2006-6-09_1-22-43,ripple_marginals_df,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-05-23 00:00:00
8,kdiba_gor01_one_2006-6-09_1-22-43,laps_time_bin_marginals_df,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-05-23 00:00:00
9,kdiba_gor01_one_2006-6-09_1-22-43,laps_marginals_df,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-05-23 00:00:00


Box(children=(Label(value='across_sessions_parsed_csv_files_path:', layout=Layout(width='auto')), HTML(value="…

In [4]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import export_across_session_CSVs

final_across_session_summary_CSVs_output_path = across_sessions_output_folder.resolve()
display(fullwidth_path_widget(final_across_session_summary_CSVs_output_path, file_name_label='final_across_session_summary_CSVs_output_path:'))
final_csv_export_paths = export_across_session_CSVs(final_output_path=final_across_session_summary_CSVs_output_path, TODAY_DAY_DATE=TODAY_DAY_DATE,
                                                    all_sessions_laps_df=all_sessions_laps_df,  all_sessions_ripple_df=all_sessions_ripple_df,  all_sessions_laps_time_bin_df=all_sessions_laps_time_bin_df,  all_sessions_ripple_time_bin_df=all_sessions_ripple_time_bin_df, 
                                                    all_sessions_simple_pearson_laps_df=all_sessions_simple_pearson_laps_df,  all_sessions_simple_pearson_ripple_df=all_sessions_simple_pearson_ripple_df,
                                                    all_sessions_all_scores_ripple_df=all_sessions_all_scores_ripple_df,  all_sessions_all_scores_laps_df=None,
                                                    )

# final_csv_export_paths: {'AcrossSession_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Laps_per-Epoch.csv'),
#  'AcrossSession_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Ripple_per-Epoch.csv'),
#  'AcrossSession_Laps_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Laps_per-TimeBin.csv'),
#  'AcrossSession_Ripple_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Ripple_per-TimeBin.csv'),
#  'AcrossSession_SimplePearson_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_SimplePearson_Laps_per-Epoch.csv'),
#  'AcrossSession_SimplePearson_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_SimplePearson_Ripple_per-Epoch.csv'),
#  'AcrossSession_AllScores_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_AllScores_Ripple_per-Epoch.csv')}

if neptuner_run is not None:
    _neptuner_run_parameters = dict(across_sessions_parsed_csv_files_path=across_sessions_parsed_csv_files_path.as_posix(), final_across_session_summary_CSVs_output_path=final_across_session_summary_CSVs_output_path.as_posix(),
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'output_files/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

    for k, v in final_csv_export_paths.items():
        neptuner_run[f"output_files/{k}"].upload(v.resolve().as_posix())
        

final_csv_export_paths

Box(children=(Label(value='final_across_session_summary_CSVs_output_path:', layout=Layout(width='auto')), HTML…

{'AcrossSession_Laps_per-Epoch': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_SimplePearson_Laps_per-Epoch': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP_AcrossSession_SimpleP

In [ ]:

if (neptuner is not None) and (neptuner_run is not None):
    neptuner.stop()

In [10]:
parsed_csv_files_df['file_type'].unique()

array(['ripple_weighted_corr_merged_df',
       'ripple_simple_pf_pearson_merged_df',
       'ripple_all_scores_merged_df', 'laps_weighted_corr_merged_df',
       'laps_simple_pf_pearson_merged_df', 'ripple_WCorrShuffle_df',
       'ripple_time_bin_marginals_df', 'ripple_marginals_df',
       'laps_time_bin_marginals_df', 'laps_marginals_df',
       'neuron_replay_stats_df'], dtype=object)

In [ ]:
parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']

### h5_files

In [5]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_h5_files

## INPUTS: h5_sessions, session_dict, cuttoff_date, known_bad_session_strs
parsed_h5_files_df, h5_contexts_paths_dict = load_across_sessions_exported_h5_files(collected_outputs_directory=collected_outputs_directory, cuttoff_date=cuttoff_date,
                                                                                    known_bad_session_strs=known_bad_session_strs)
h5_session_contexts = list(h5_contexts_paths_dict.keys())
included_h5_paths = list(h5_contexts_paths_dict.values())

if neptuner_run is not None:
    _neptuner_run_parameters = dict(parsed_h5_files_df=parsed_h5_files_df,
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

parsed_h5_files_df
# h5_contexts_paths_dict


collected_outputs_directory: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs


,session,file_type,path,decoding_time_bin_size_str,export_datetime
0,fet11-01_12-58-54,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
1,2006-6-12_16-53-46,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
2,2006-6-12_15-55-31,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
3,2006-6-09_1-22-43,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
4,2006-6-08_21-16-25,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
5,2006-6-08_14-26-15,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
6,2006-6-07_16-40-19,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
7,2006-4-10_12-58-3,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
8,2006-4-10_12-25-50,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12
9,2006-4-09_17-29-30,pipeline_results,/Users/pho/University of Michigan Dropbox/MED-...,None,2024-06-12


In [6]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

included_session_contexts = deepcopy(h5_session_contexts)
included_h5_paths = deepcopy(included_h5_paths)
num_sessions = len(included_session_contexts)
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths,
                                                                                                                                                    override_output_parent_path=across_sessions_output_folder, output_path_suffix=f'{TODAY_DAY_DATE}',
                                                                                                                                                    should_restore_native_column_types=True, include_csv=True, include_pkl=True)


if neptuner_run is not None:
    _neptuner_run_parameters = dict(neuron_identities_table=neuron_identities_table, long_short_fr_indicies_analysis_table=long_short_fr_indicies_analysis_table, neuron_replay_stats_table=neuron_replay_stats_table,
                                       num_sessions=num_sessions)
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

    for output_name, a_paths_dict in output_path_dicts.items():
        for format_extension, an_output_path in a_paths_dict.items():
            neptuner_run[f"output_files/{format_extension}/{output_name}"].upload(an_output_path.resolve().as_posix())


# {'neuron_identities_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_identities_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_identities_table.pkl')},
#  'long_short_fr_indicies_analysis_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/long_short_fr_indicies_analysis_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/long_short_fr_indicies_analysis_table.pkl')},
#  'neuron_replay_stats_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_replay_stats_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_replay_stats_table.pkl')}}

output_path_dicts

failed for file path: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee_2006-6-12_16-53-46_pipeline_results.h5, table_key: /kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table``. Skipping.
failed for file path: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee_11-02_17-46-44_pipeline_results.h5, table_key: /kdiba/pin01/one/11-02_17-46-44/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-02_17-46-44/neuron_identities/table``. Skipping.
concatenating dataframes from 12 of 14 files
failed for file path: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee_2006-6-12_16-53-46_pipeline_results.h5, table_key: /kdib

{'neuron_identities_table': {'.csv': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP/neuron_identities_table.csv'),
  '.pkl': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP/neuron_identities_table.pkl')},
 'long_short_fr_indicies_analysis_table': {'.csv': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP/long_short_fr_indicies_analysis_table.csv'),
  '.pkl': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP/long_short_fr_indicies_analysis_table.pkl')},
 'neuron_replay_stats_table': {'.csv': PosixPath('/Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/across_sessions/2024-06-12_rMBP/neuron_replay_stats_table.csv'),
  '.pkl': PosixPath('/User

In [11]:
neuron_replay_stats_table

,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,neuron_type,aclu,custom_frs_index,is_rate_extrema,is_refined_exclusive,is_refined_LxC,is_refined_SxC,is_long_peak_left_cap,is_long_peak_right_cap,is_long_peak_either_cap,LS_pf_peak_x_diff,long_LR_pf2D_peak_x,long_LR_pf2D_peak_y,long_RL_pf2D_peak_x,long_RL_pf2D_peak_y,short_LR_pf2D_peak_x,short_LR_pf2D_peak_y,short_RL_pf2D_peak_x,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_uid,neuron_uid,session_datetime
0,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,SHARED,0.41242440259338126,0.11553338629183138,-0.2968910163015499,7.912739600903586,nan,nan,4.162582001748484,nan,nan,45,45,0,pyr,2,0.2334295442830472,False,False,False,False,False,False,False,nan,212.155527,138.787910,82.294630,145.066311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|2,2006-06-08 14:26:15
1,kdiba,gor01,one,2006-6-08_14-26-15,141.385524,198.280871,SHARED,0.2740687245121229,1.3945946124809006,1.1205258879687776,9.241991017956833,8.5362581427967,-0.7057328751601322,4.758029871234478,4.965426377638801,0.20739650640432306,91,23,68,pyr,3,-0.6929576266509063,True,False,False,False,False,False,False,-56.8953462153255,140.520949,138.781664,NaN,NaN,121.601684,138.785344,195.336656,142.575786,143.282036,NaN,128.109944,NaN,-15.172092,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|3,2006-06-08 14:26:15
2,kdiba,gor01,one,2006-6-08_14-26-15,NaN,179.315755,RIGHT_ONLY,0.11938205363580597,0.9248959624728122,0.8055139088370062,nan,12.484800687765516,nan,nan,6.704848325119164,nan,87,0,87,pyr,4,-0.9825415552245808,True,True,False,True,False,False,False,nan,NaN,NaN,NaN,NaN,NaN,NaN,177.874096,144.799775,NaN,NaN,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|4,2006-06-08 14:26:15
3,kdiba,gor01,one,2006-6-08_14-26-15,99.662271,202.073894,SHARED,2.3260457715375873,2.0126115307154993,-0.31343424082208804,11.994903850060455,11.315249040961577,-0.6796548090988779,7.1604748107990215,6.663930285838538,-0.4965445249604832,229,126,103,pyr,5,-0.035628934583543,False,False,False,False,False,False,False,-102.41162318758592,234.462951,138.805607,97.174785,140.858022,199.777578,144.348824,111.319822,140.813779,234.314590,93.972736,200.177382,109.144828,-34.137208,15.172092,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|5,2006-06-08 14:26:15
4,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,SHARED,25.389639299524944,12.75463574941881,-12.635003550106134,105.81199584799322,41.36496231876168,-64.44703352923153,65.60081757375909,27.059799034090243,-38.54101853966884,602,345,257,intr,6,0.09923680577542646,False,False,False,False,False,False,False,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|6,2006-06-08 14:26:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,kdiba,pin01,one,fet11-01_12-58-54,180.571251,176.844138,SHARED,1.2275172634201952,1.3169564137781686,0.08943915035797345,8.804706579928968,9.155474207048053,0.35076762711908493,5.0161119216745815,5.236215310413111,0.2201033887385293,181,100,81,pyr,28,-0.08756873199979255,False,False,False,False,False,False,False,3.7271132894561845,175.339442,141.003729,186.412849,140.908855,176.615749,140.846341,142.356312,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28,2009-11-01 12:58:54
686,kdiba,pin01,one,fet11-01_12-58-54,

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults

# matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_directory, BATCH_DATE_TO_USE=TODAY_DAY_DATE, plotting_enabled=True)

In [ ]:
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False)

In [13]:
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions) ## WORKS


register_output_file(output_path: /Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/SCRATCH/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2024-06-12/kdiba/{}_14_across_sessions_firing_rate_index.png, ...)
	 saved "file:///Users/pho/repo/Pho%20Secondary%20Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/SCRATCH/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2024-06-12/kdiba/%7B%7D_14_across_sessions_firing_rate_index.png"


In [ ]:
print(list(all_sessions_all_scores_ripple_df.columns)) # ['start', 'stop', 'label', 'duration', 'is_user_annotated_epoch', 'is_valid_epoch', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'P_Long_LR', 'score_long_LR', 'velocity_long_LR', 'intercept_long_LR', 'speed_long_LR', 'wcorr_long_LR', 'pearsonr_long_LR', 'travel_long_LR', 'coverage_long_LR', 'jump_long_LR', 'longest_sequence_length_ratio_long_LR', 'direction_change_bin_ratio_long_LR', 'congruent_dir_bins_ratio_long_LR', 'total_congruent_direction_change_long_LR', 'P_Long_RL', 'score_long_RL', 'velocity_long_RL', 'intercept_long_RL', 'speed_long_RL', 'wcorr_long_RL', 'pearsonr_long_RL', 'travel_long_RL', 'coverage_long_RL', 'jump_long_RL', 'longest_sequence_length_ratio_long_RL', 'direction_change_bin_ratio_long_RL', 'congruent_dir_bins_ratio_long_RL', 'total_congruent_direction_change_long_RL', 'P_Short_LR', 'score_short_LR', 'velocity_short_LR', 'intercept_short_LR', 'speed_short_LR', 'wcorr_short_LR', 'pearsonr_short_LR', 'travel_short_LR', 'coverage_short_LR', 'jump_short_LR', 'longest_sequence_length_ratio_short_LR', 'direction_change_bin_ratio_short_LR', 'congruent_dir_bins_ratio_short_LR', 'total_congruent_direction_change_short_LR', 'P_Short_RL', 'score_short_RL', 'velocity_short_RL', 'intercept_short_RL', 'speed_short_RL', 'wcorr_short_RL', 'pearsonr_short_RL', 'travel_short_RL', 'coverage_short_RL', 'jump_short_RL', 'longest_sequence_length_ratio_short_RL', 'direction_change_bin_ratio_short_RL', 'congruent_dir_bins_ratio_short_RL', 'total_congruent_direction_change_short_RL', 'ripple_start_t', 'long_best_travel', 'short_best_travel', 'travel_diff', 'long_best_coverage', 'short_best_coverage', 'coverage_diff', 'long_best_jump', 'short_best_jump', 'jump_diff', 'long_best_longest_sequence_length_ratio', 'short_best_longest_sequence_length_ratio', 'longest_sequence_length_ratio_diff', 'long_best_direction_change_bin_ratio', 'short_best_direction_change_bin_ratio', 'direction_change_bin_ratio_diff', 'long_best_congruent_dir_bins_ratio', 'short_best_congruent_dir_bins_ratio', 'congruent_dir_bins_ratio_diff', 'long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change', 'total_congruent_direction_change_diff', 'session_name', 'time_bin_size', 'delta_aligned_start_t']
# ['long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change']
['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']

In [ ]:
all_sessions_simple_pearson_ripple_df # 3138 rows × 24 columns

In [ ]:
all_sessions_simple_pearson_laps_df # 931 rows × 25 columns

In [ ]:
all_sessions_ripple_df # 17592 rows × 10 columns

In [ ]:
csv_files

In [ ]:
all_sessions_all_scores_ripple_df

In [ ]:

all_sessions_simple_pearson_ripple_df

In [ ]:
## find all columns starting with 'wcorr': wcorr_long_LR
sub_string: str = 'wcorr'
sub_string: str = 'pearsonr'
columns_list = list(all_sessions_simple_pearson_ripple_df.columns)
matching_columns = [s for s in columns_list if sub_string in s]
print(matching_columns) # ['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'pearsonr_abs_diff']
['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

assert np.shape(all_sessions_simple_pearson_ripple_df)[0] == np.shape(all_sessions_all_scores_ripple_df)[0], f"np.shape(all_sessions_all_scores_ripple_df)[0]: {np.shape(all_sessions_all_scores_ripple_df)[0]} != np.shape(all_sessions_simple_pearson_ripple_df)[0]: {np.shape(all_sessions_simple_pearson_ripple_df)[0]}"



In [ ]:
['long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'pearsonr_abs_diff']
['wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff']

# 2024-03-02 - Get only the user-annotated ripples:

In [7]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import _split_user_annotated_ripple_df

## Bump
# input_df = all_sessions_simple_pearson_ripple_df
# input_df = all_sessions_all_scores_ripple_df

all_sessions_all_scores_ripple_df, (valid_ripple_df, invalid_ripple_df), (user_approved_ripple_df, user_rejected_ripple_df) = _split_user_annotated_ripple_df(all_sessions_all_scores_ripple_df)

## 2024-03-14 - 'is_valid_epoch' column
# 'is_valid_epoch'
## OUTPUTS: valid_ripple_df, invalid_ripple_df, user_approved_ripple_df, user_rejected_ripple_df, (user_annotated_epoch_unique_session_names, unannotated_session_names)
user_approved_ripple_df

user_annotated_epoch_unique_session_names: ['kdiba_gor01_one_2006-6-09_1-22-43']
unannotated_session_names: set()
n_input_df_rows: 412
	 n_valid_df_rows: 136
	 n_invalid_df_rows: 276
n_input_df_rows: 136
	 n_user_approved_df_rows: 39
	 n_user_rejected_df_rows: 97


,start,stop,label,duration,P_LR,P_RL,P_Long,P_Short,P_Long_LR,score_long_LR,velocity_long_LR,intercept_long_LR,speed_long_LR,wcorr_long_LR,pearsonr_long_LR,travel_long_LR,coverage_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,direction_change_bin_ratio_long_LR,congruent_dir_bins_ratio_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,integral_second_derivative_long_LR,stddev_of_diff_long_LR,P_Long_RL,score_long_RL,velocity_long_RL,intercept_long_RL,speed_long_RL,wcorr_long_RL,pearsonr_long_RL,travel_long_RL,coverage_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,direction_change_bin_ratio_long_RL,congruent_dir_bins_ratio_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,integral_second_derivative_long_RL,stddev_of_diff_long_RL,P_Short_LR,score_short_LR,velocity_short_LR,intercept_short_LR,speed_short_LR,wcorr_short_LR,pearsonr_short_LR,travel_short_LR,...,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_name,time_bin_size,delta_aligned_start_t,pre_post_delta_category
0,105.400143,105.562560,8,0.162417,0.997449,0.002551,0.827140,0.172860,0.825030,0.235594,-981.0841003490076,-103383.35970988029,981.0841003490076,0.809407,0.842463,0.114597,0.368421,0.001752,0.300000,0.333333,0.666667,190.270856,220.714193,17377.439317,30.071090,2.109962e-03,0.466703,-0.0,42.78551541859196,0.0,0.546726,0.604015,0.280566,0.280702,0.003672,0.700000,0.333333,0.555556,270.184615,540.369231,2.172035e+05,85.581942,0.172419,0.355220,-660.662693837691,-69569.8043740628,660.662693837691,0.602887,0.775503,0.152686,...,0.076904,-0.0,-0.0,0.0,42.78551541859196,84.64510370014965,-41.859588281557684,0.0,0.0,0.0,0.546726,0.398772,0.147955,0.604015,0.540470,0.063545,0.280566,0.284819,-0.004253,0.280702,0.245614,0.035088,0.003672,0.003596,0.000075,0.700000,0.700000,0.000000,0.333333,0.333333,0.000000,0.555556,0.333333,0.222222,270.184615,186.465439,83.719177,540.369231,369.125460,171.243770,217203.510259,88407.722524,128795.787736,85.581942,61.738114,23.843828,kdiba_gor01_one_2006-6-09_1-22-43,0.025,-923.916466,pre-delta
1,132.511389,132.791003,17,0.279613,0.229219,0.770781,0.971456,0.028544,0.222676,0.308273,-33.97693854022707,-4258.130527597626,33.97693854022707,-0.807946,-0.754849,0.174489,0.245614,0.003755,0.294118,0.312500,0.437500,380.541712,597.450487,174759.114728,63.970617,7.487800e-01,0.262468,222.97365917022896,29792.8057957203,222.97365917022896,-0.890005,-0.835692,0.076686,0.333333,0.001502,0.647059,0.250000,0.687500,232.130444,262.573781,6.531021e+03,22.013884,0.006543,0.384295,29.729821222697268,4150.0016249617565,29.729821222697268,-0.641119,-0.621914,0.247749,...,-0.076022,-33.97693854022707,29.729821222697268,4.247117317529799,-4258.130527597626,4150.0016249617565,108.12890263586905,33.97693854022707,29.729821222697268,4.247117317529799,-0.807946,-0.641119,0.166827,-0.754849,-0.621914,0.132936,0.174489,0.247749,-0.073259,0.245614,0.245

### 2024-02-29 - 4pm - Filter the events for those meeting wcorr criteria:


In [14]:
# df: pd.DataFrame = deepcopy(all_sessions_user_annotated_ripple_df)
df: pd.DataFrame = deepcopy(valid_ripple_df) # valid epochs, but not just those that the user approved
# df: pd.DataFrame = deepcopy(user_approved_ripple_df)

## INPUTS: df

min_wcorr_threshold: float = 0.33
min_wcorr_diff_threshold: float = 0.2

# is_included_large_wcorr_diff = np.any((df[['wcorr_abs_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_large_wcorr_diff = np.any((df[['wcorr_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_high_wcorr = np.any((df[['long_best_wcorr', 'short_best_wcorr']].abs() > min_wcorr_threshold), axis=1)

df = df[is_included_high_wcorr]
df

# wcorr_long_LR


,start,stop,label,duration,is_user_annotated_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,score_long_LR,velocity_long_LR,intercept_long_LR,speed_long_LR,wcorr_long_LR,pearsonr_long_LR,travel_long_LR,coverage_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,direction_change_bin_ratio_long_LR,congruent_dir_bins_ratio_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,integral_second_derivative_long_LR,stddev_of_diff_long_LR,P_Long_RL,score_long_RL,velocity_long_RL,intercept_long_RL,speed_long_RL,wcorr_long_RL,pearsonr_long_RL,travel_long_RL,coverage_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,direction_change_bin_ratio_long_RL,congruent_dir_bins_ratio_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,integral_second_derivative_long_RL,stddev_of_diff_long_RL,P_Short_LR,score_short_LR,velocity_short_LR,intercept_short_LR,speed_short_LR,wcorr_short_LR,pearsonr_short_LR,...,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_name,time_bin_size,delta_aligned_start_t,pre_post_delta_category
0,93.027055,93.465245,4,0.438190,False,0.554979,0.445021,0.507392,0.492608,0.281592,0.182291,-0.0,244.47262259336992,0.0,0.507870,0.589175,0.309823,0.368421,0.004172,0.222222,0.500000,0.500000,939.938028,1723.853954,637694.098126,95.265151,0.225800,0.181924,-0.0,248.27803970987515,0.0,0.496553,0.627194,0.409677,0.333333,0.004673,0.222222,0.576923,0.384615,1145.430552,2279.444853,1.017999e+06,119.815227,0.273387,0.178224,-9.147637299291477,-644.6324886551572,9.147637299291477,0.392682,0.474401,...,0.008275,-0.0,-91.47637299291532,-91.47637299291532,248.27803970987515,-8330.728869222017,-8082.450829512142,0.0,91.47637299291532,-91.47637299291532,0.496553,0.377063,0.119489,0.627194,0.463406,0.163788,0.409677,0.278495,0.131183,0.333333,0.368421,-0.035088,0.004673,0.005828,-0.001155,0.222222,0.259259,-0.037037,0.576923,0.384615,0.192308,0.384615,0.346154,0.038462,1145.430552,570.812567,574.617985,2279.444853,1042.684290,1236.760563,1.017999e+06,264151.558813,753847.798758,119.815227,62.689511,57.125716,kdiba_gor01_one_2006-6-09_1-22-43,0.025,-936.289554,pre-delta
1,105.400143,105.562560,8,0.162417,True,0.997449,0.002551,0.827140,0.172860,0.825030,0.235594,-981.0841003490076,-103383.35970988029,981.0841003490076,0.809407,0.842463,0.114597,0.368421,0.001752,0.300000,0.333333,0.666667,190.270856,220.714193,17377.439317,30.071090,0.002110,0.466703,-0.0,42.78551541859196,0.0,0.546726,0.604015,0.280566,0.280702,0.003672,0.700000,0.333333,0.555556,270.184615,540.369231,2.172035e+05,85.581942,0.172419,0.355220,-660.662693837691,-69569.8043740628,660.662693837691,0.602887,0.775503,...,0.076904,-0.0,-0.0,0.0,42.78551541859196,84.64510370014965,-41.859588281557684,0.0,0.0,0.0,0.546726,0.398772,0.147955,0.604015,0.540470,0.063545,0.280566,0.284819,-0.004253,0.280702,0.245614,0.035088,0.003672,0.003596,0.000075,0.700000,0.700000,0.000000,0.333333,0.333333,0.00000

In [15]:
all_sessions_all_scores_ripple_df.time_bin_size.unique()

array([0.025])

In [16]:
all_sessions_ripple_time_bin_df

,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,delta_aligned_start_t,time_bin_size,pre_post_delta_category
0,3.840469e-01,6.159531e-01,6.323099e-01,3.676901e-01,0,221.353593,kdiba_gor01_one_2006-6-09_1-22-43,-807.963016,0.03,pre-delta
1,3.115718e-01,6.884282e-01,7.989193e-01,2.010807e-01,0,221.383593,kdiba_gor01_one_2006-6-09_1-22-43,-807.933016,0.03,pre-delta
2,5.370012e-01,4.629988e-01,4.744944e-01,5.255056e-01,0,221.413593,kdiba_gor01_one_2006-6-09_1-22-43,-807.903016,0.03,pre-delta
3,7.786691e-01,2.213309e-01,3.977340e-01,6.022660e-01,0,221.443593,kdiba_gor01_one_2006-6-09_1-22-43,-807.873016,0.03,pre-delta
4,4.055792e-01,5.944208e-01,7.480698e-01,2.519302e-01,0,221.473593,kdiba_gor01_one_2006-6-09_1-22-43,-807.843016,0.03,pre-delta
...,...,...,...,...,...,...,...,...,...,...
289,2.214251e-07,9.999998e-01,2.214252e-07,9.999998e-01,3,753.510995,kdiba_gor01_one_2006-6-09_1-22-43,-275.805613,0.50,pre-delta
290,2.235056e-07,9.999998e-01,1.000000e+00,6.958079e-79,4,805.550925,kdiba_gor01_one_2006-6-09_1-22-43,-223.765683,0.50,pre-delta
291,1.000000e+00,8.854056e-33,7.282934e-01,2.717066e-01,5,1022.036743,kdiba_gor01_one_2006-6-09_1-22-43,-7.279865,0.50,pre-delta
292,9.981089e-01,1.891067e-03,6.094575e-17,1.000000e+00,6,1208.839776,kdiba_gor01_one_2006-6-09_1-22-43,179.523167,0.50,post-delta


### Single-time bin size version:

In [17]:
from neuropy.utils.misc import add_explicit_dataframe_columns_from_lookup_df


# target_time_bin_size: float = 0.005 # 0.025 # 0.08222222

target_time_bin_size: float = 0.025

## INPUTS: all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)

def build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df, all_sessions_epochs_df, all_sessions_epochs_time_bin_df, target_time_bin_size: float, included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch']):
    """ Filters the epochs dataframe down to a single time_bin_size specified by `target_time_bin_size`.  """
    print(f'all_sessions_ripple_df.time_bin_size.unique(): {all_sessions_epochs_df.time_bin_size.unique()}')
    single_time_bin_size_all_sessions_epochs_df = deepcopy(all_sessions_epochs_df[np.isclose(all_sessions_epochs_df['time_bin_size'], target_time_bin_size)])
    print(f'np.shape(single_time_bin_size_all_sessions_ripple_df): {np.shape(single_time_bin_size_all_sessions_epochs_df)}')

    print(f'all_sessions_ripple_time_bin_df.time_bin_size.unique(): {all_sessions_epochs_time_bin_df.time_bin_size.unique()}')
    single_time_bin_size_all_sessions_epochs_time_bin_df = deepcopy(all_sessions_epochs_time_bin_df[np.isclose(all_sessions_epochs_time_bin_df['time_bin_size'], target_time_bin_size)])
    print(f'np.shape(single_time_bin_size_all_sessions_ripple_time_bin_df): {np.shape(single_time_bin_size_all_sessions_epochs_time_bin_df)}')

    # single_time_bin_size_all_sessions_ripple_time_bin_df
    # single_time_bin_size_all_sessions_ripple_df # has ['ripple_start_t']
    # all_sessions_all_scores_ripple_df

    ## recover the important columns (user-annotation, epoch validity) from the newer `all_sessions_all_scores_ripple_df` for use in 'single_time_bin_size_all_sessions_ripple_df'
    all_sessions_all_scores_epochs_df['delta_aligned_start_t'] = all_sessions_all_scores_epochs_df['delta_aligned_start_t'].astype(float)
    single_time_bin_size_all_sessions_epochs_df['delta_aligned_start_t'] = single_time_bin_size_all_sessions_epochs_df['delta_aligned_start_t'].astype(float)

    # Added 'delta_aligned_start_t' for the merge
    single_time_bin_size_all_sessions_epochs_df = add_explicit_dataframe_columns_from_lookup_df(single_time_bin_size_all_sessions_epochs_df, all_sessions_all_scores_epochs_df[included_columns], join_column_name='delta_aligned_start_t')
    single_time_bin_size_all_sessions_epochs_df.sort_values(by=['delta_aligned_start_t'], inplace=True) # Need to re-sort by timestamps once done
    single_time_bin_size_all_sessions_epochs_df

    single_time_bin_size_all_sessions_epochs_time_bin_df = add_explicit_dataframe_columns_from_lookup_df(single_time_bin_size_all_sessions_epochs_time_bin_df, all_sessions_all_scores_epochs_df[included_columns], join_column_name='delta_aligned_start_t')
    single_time_bin_size_all_sessions_epochs_time_bin_df.sort_values(by=['t_bin_center'], inplace=True) # Need to re-sort by timestamps once done
    
    ## Add plotly helper columns:
    for a_df in (all_sessions_all_scores_epochs_df, all_sessions_epochs_df, all_sessions_epochs_time_bin_df, single_time_bin_size_all_sessions_epochs_df, single_time_bin_size_all_sessions_epochs_time_bin_df):
        a_df['pre_post_delta_category'] = 'post-delta'
        a_df['pre_post_delta_category'][a_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'

    ## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df
    return single_time_bin_size_all_sessions_epochs_df, single_time_bin_size_all_sessions_epochs_time_bin_df


single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df = build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df=all_sessions_all_scores_ripple_df,
                                                                                                            all_sessions_epochs_df=all_sessions_ripple_df, all_sessions_epochs_time_bin_df=all_sessions_ripple_time_bin_df, target_time_bin_size=0.005,
                                                                                                            included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch'])

                                                                                                            
single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df = build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df=all_sessions_simple_pearson_laps_df,
                                                                                                            all_sessions_epochs_df=all_sessions_laps_df, all_sessions_epochs_time_bin_df=all_sessions_laps_time_bin_df, target_time_bin_size=0.005,
                                                                                                            included_columns = ['delta_aligned_start_t',])

## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df
## OUTPUTS: single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df 

single_time_bin_size_all_sessions_laps_df


# # Recover the important columns from the newer `all_sessions_all_scores_ripple_df`
# included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch']  # Added 'delta_aligned_start_t' for the merge
# single_time_bin_size_all_sessions_ripple_df = pd.merge(single_time_bin_size_all_sessions_ripple_df, 
#                      all_sessions_all_scores_ripple_df[included_columns], 
#                      on='delta_aligned_start_t', 
#                      how='left')


# single_time_bin_size_all_sessions_ripple_df
## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_df
# single_time_bin_size_all_sessions_ripple_time_bin_df
all_sessions_simple_pearson_laps_df
single_time_bin_size_all_sessions_laps_time_bin_df

all_sessions_ripple_df.time_bin_size.unique(): [0.03       0.08222222 0.13444444 0.18666667 0.23888889 0.29111111
 0.34333333 0.39555556 0.44777778 0.5       ]
np.shape(single_time_bin_size_all_sessions_ripple_df): (0, 11)
all_sessions_ripple_time_bin_df.time_bin_size.unique(): [0.03       0.08222222 0.13444444 0.18666667 0.23888889 0.29111111
 0.34333333 0.39555556 0.44777778 0.5       ]
np.shape(single_time_bin_size_all_sessions_ripple_time_bin_df): (0, 10)
all_sessions_ripple_df.time_bin_size.unique(): [0.03       0.08222222 0.13444444 0.18666667 0.23888889 0.29111111
 0.34333333 0.39555556 0.44777778 0.5       ]
np.shape(single_time_bin_size_all_sessions_ripple_df): (0, 11)
all_sessions_ripple_time_bin_df.time_bin_size.unique(): [0.03       0.08222222 0.13444444 0.18666667 0.23888889 0.29111111
 0.34333333 0.39555556 0.44777778 0.5       ]
np.shape(single_time_bin_size_all_sessions_ripple_time_bin_df): (0, 10)


/var/folders/c7/74vbnwdj7531qy2glkcjq6jr0000gn/T/ipykernel_55388/2552745437.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df['pre_post_delta_category'][a_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'
/var/folders/c7/74vbnwdj7531qy2glkcjq6jr0000gn/T/ipykernel_55388/2552745437.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df['pre_post_delta_category'][a_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'


,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,time_bin_size,pre_post_delta_category,delta_aligned_start_t


In [18]:
all_sessions_all_scores_ripple_df.delta_aligned_start_t.nunique()
all_sessions_all_scores_ripple_df.start.nunique()
all_sessions_all_scores_ripple_df

,start,stop,label,duration,is_user_annotated_epoch,is_valid_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,score_long_LR,velocity_long_LR,intercept_long_LR,speed_long_LR,wcorr_long_LR,pearsonr_long_LR,travel_long_LR,coverage_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,direction_change_bin_ratio_long_LR,congruent_dir_bins_ratio_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,integral_second_derivative_long_LR,stddev_of_diff_long_LR,P_Long_RL,score_long_RL,velocity_long_RL,intercept_long_RL,speed_long_RL,wcorr_long_RL,pearsonr_long_RL,travel_long_RL,coverage_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,direction_change_bin_ratio_long_RL,congruent_dir_bins_ratio_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,integral_second_derivative_long_RL,stddev_of_diff_long_RL,P_Short_LR,score_short_LR,velocity_short_LR,intercept_short_LR,speed_short_LR,wcorr_short_LR,...,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_name,time_bin_size,delta_aligned_start_t,pre_post_delta_category
0,42.658077,42.690456,0,0.032379,False,False,3.377521e-03,0.996622,0.968932,0.031068,3.272587e-03,0.256083,-4043.2556862872684,-172341.49561295123,4043.2556862872684,0.724297,0.252205,0.408993,0.385965,0.001919,1.000000,0.000000,1.000000,87.524594,87.524594,0.000000,0.000000,0.965659,0.240893,-237.83856978160279,-9952.636617169057,237.83856978160279,-0.515537,-0.125155,0.533470,0.315789,0.002503,1.000000,0.000000,1.000000,114.162513,114.162513,0.000000e+00,0.000000,1.049340e-04,0.348944,-951.3542791264147,-40403.163903145796,951.3542791264147,0.294705,...,-0.092860,-4043.2556862872684,-951.3542791264147,3091.9014071608535,-172341.49561295123,-40403.163903145796,131938.33170980544,4043.2556862872684,951.3542791264147,3091.9014071608535,0.724297,0.294705,0.429593,0.252205,0.178567,0.073638,0.408993,0.079280,0.329714,0.385965,0.228070,0.157895,0.001919,0.000372,0.001547,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,87.524594,11.416251,76.108342,87.524594,11.416251,76.108342,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,kdiba_gor01_one_2006-6-09_1-22-43,0.025,-986.658531,pre-delta
1,55.723809,55.897899,1,0.174090,False,False,6.847150e-01,0.315285,0.480394,0.519606,3.289331e-01,0.128989,79.27952326052683,4506.841761602025,79.27952326052683,0.173317,0.152431,0.254880,0.403509,0.003922,0.300000,0.555556,0.333333,251.157530,490.898808,128781.306536,78.542924,0.151461,0.163663,564.8666032312468,31592.320614111068,564.8666032312468,-0.141276,0.066632,0.296372,0.333333,0.004673,0.500000,0.333333,0.222222,388.152546,570.812567,1.710809e+05,97.482741,3.557819e-01,0.178470,184.98555427456267,10413.25175111521,184.98555427456267,0.093641,...,-0.042692,564.8666032312468,105.70603101403574,459.1605722172111,31592.320614111068,5979.6388418638135,25612.681772247255,564.8666032312468,105.70603101403574,459.1605722172111,-

In [19]:
single_time_bin_size_all_sessions_ripple_time_bin_df.delta_aligned_start_t.nunique()

0

In [14]:
print(all_sessions_ripple_time_bin_df.delta_aligned_start_t.nunique())
print(all_sessions_ripple_time_bin_df.t_bin_center.nunique())

261
261


# Plotting via Plotly
`!pip install kaleido=="v0.1.0.post1" `

In [20]:
import plotly.io as pio
from pyphoplacecellanalysis.Pho2D.plotly.plotly_templates import PlotlyHelpers
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_stacked_histograms
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# fig_size_kwargs = {'width': 1650, 'height': 480}
resolution_multiplier = 2
fig_size_kwargs = {'width': resolution_multiplier*1650, 'height': resolution_multiplier*480}
is_dark_mode, template = PlotlyHelpers.get_plotly_template(is_dark_mode=False)
pio.templates.default = template

# figure_export_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Presentations\2024-05-30 - Pho iNAV Poster\Figures').resolve()
figure_export_path = Path('/Users/pho/Dropbox (Personal)/Active/Kamran Diba Lab/Presentations/2024-05-30 - Pho iNAV Poster/Figures').resolve()


assert figure_export_path.exists()

from PIL import Image

from pyphocorehelpers.programming_helpers import copy_image_to_clipboard

def save_plotly(a_fig, a_fig_context):
    """ 
    captures: TODAY_DAY_DATE, figures_folder, neptuner_run
    """
    fig_save_path: Path = figures_folder.joinpath('_'.join([TODAY_DAY_DATE, sanitize_filename_for_Windows(a_fig_context.get_description())])).resolve()
    figure_out_paths = {'.html': fig_save_path.with_suffix('.html'), '.png': fig_save_path.with_suffix('.png')}
    a_fig.write_html(figure_out_paths['.html'])
    display(fullwidth_path_widget(figure_out_paths['.html'], file_name_label='.html'))
    # print(file_uri_from_path(figure_out_paths['.html']))
    a_fig.write_image(figure_out_paths['.png'])
    # print(file_uri_from_path(figure_out_paths['.png']))
    display(fullwidth_path_widget(figure_out_paths['.png'], file_name_label='.png'))

    if neptuner_run is not None:
        a_full_figure_path_key: str = a_fig_context.get_description(separator='/', include_property_names=True, key_value_separator=':') # .replace(' ', '_')
        # a_full_figure_path_key: str = a_fig_context.get_description(separator=':', include_property_names=True, key_value_separator='|')
        print(f'a_full_figure_path_key: "{a_full_figure_path_key}"')
        # neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_fig)
        neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(figure_out_paths['.html'].as_posix())
        # neptuner.figures[f"{a_full_figure_path_key}"].upload(a_fig)
        
    return figure_out_paths


In [21]:
## INPUTS: all_sessions_all_scores_ripple_df
# concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df).infer_objects()
concatenated_ripple_df = deepcopy(single_time_bin_size_all_sessions_ripple_df).infer_objects()

# single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df

concatenated_ripple_df

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,session_name,time_bin_size,epoch_idx,pre_post_delta_category,delta_aligned_start_t,is_user_annotated_epoch,is_valid_epoch


In [22]:


ripple_title_prefix = "User Annotated Sessions Ripples"

# concatenated_ripple_df = deepcopy(user_approved_ripple_df)
# ripple_title_prefix = "User Selected Ripples Only"

# concatenated_ripple_df = deepcopy(user_rejected_ripple_df)
# ripple_title_prefix = "User Rejected Ripples Only"

# concatenated_ripple_df = deepcopy(df)


# Create a bubble chart for ripples
enabled_time_bin_sizes = None
ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
fig_ripples, figure_ripples_context = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=25, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=enabled_time_bin_sizes, main_plot_mode='default', title=ripple_title, is_dark_mode=is_dark_mode)
figure_ripples_context = figure_ripples_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='hist', comparison='pre-post-delta')
fig_ripples = fig_ripples.update_layout(fig_size_kwargs)
fig_ripples.show()
# fig_to_clipboard(fig_ripples)
figure_out_paths = save_plotly(a_fig=fig_ripples, a_fig_context=figure_ripples_context)

num_unique_sessions: 0, num_unique_time_bins: 0
main_plot_mode: default
num_cols: 2


Box(children=(Label(value='.html', layout=Layout(width='auto')), HTML(value="<b style='font-size: larger;'>/Us…

ValueError: Failed to start Kaleido subprocess. Error stream:

/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/.venv/lib/python3.9/site-packages/kaleido/executable/kaleido: line 4: cd: /Users/pho/repo/Pho: No such file or directory
/Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/.venv/lib/python3.9/site-packages/kaleido/executable/kaleido: line 5: ./bin/kaleido: No such file or directory


In [ ]:
histogram_bins = 25
# concatenated_ripple_df = deepcopy(df)

# ripple_title_prefix = 'test'
# # Create a bubble chart for ripples
# ripple_num_unique_sessions: int = concatenated_ripple_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
# # ripple_num_unique_time_bins: int = concatenated_ripple_df.time_bin_size.nunique(dropna=True)
# ripple_title_string_suffix: str = f'{ripple_num_unique_sessions} Sessions'
# ripple_title: str = f"{ripple_title_prefix} - {ripple_title_string_suffix}"
# fig_ripples = _helper_build_figure(data_results_df=concatenated_ripple_df, histogram_bins=histogram_bins, earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end, enabled_time_bin_sizes=None,
#                                     main_plot_mode='separate_row_per_session', title=ripple_title)

# concatenated_ripple_df = deepcopy(concatenated_ripple_df)
# concatenated_ripple_df = deepcopy(all_sessions_ripple_df)
# concatenated_ripple_df = deepcopy(all_sessions_ripple_time_bin_df)
concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)
# concatenated_ripple_df = deepcopy(single_time_bin_size_all_sessions_ripple_df)
concatenated_ripple_df
# ['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']


# variable_name = 'P_Long'
variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
# variable_name = 'long_best_pf_peak_x_pearsonr'
# variable_name = 'long_best_jump'
# variable_name = 'wcorr_abs_diff'
# variable_name = 'pearsonr_abs_diff'
# variable_name = 'direction_change_bin_ratio_diff'
# variable_name = 'longest_sequence_length_ratio_diff'
# variable_name = 'long_best_longest_sequence_length_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'congruent_dir_bins_ratio_diff'
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 

px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}

px_scatter_kwargs.pop('color')

# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
hist_kwargs.pop('color')
# new_fig_ripples, new_fig_ripples_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
#                         px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
#                         time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text='Is User Selected', is_dark_mode=is_dark_mode)


new_fig_ripples, new_fig_ripples_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text=None, is_dark_mode=is_dark_mode)

                        

new_fig_ripples = new_fig_ripples.update_layout(fig_size_kwargs)

_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')

# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)
new_fig_ripples_context = new_fig_ripples_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=new_fig_ripples_context)
new_fig_ripples

In [ ]:
new_fig_ripples_context

In [ ]:
# variable_name = 'short_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
variable_name = 'short_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"pre_post_delta_category", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], , 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="pre_post_delta_category") # , histnorm='probability density'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples.show()
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=figure_context)


In [ ]:
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_total_congruent_direction_change'
variable_name = 'wcorr_diff'
# variable_name = 'long_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'", 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples.show()

figure_context = figure_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=figure_context)


## Stats Tests

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import _perform_stats_tests


# stats_variable_name: str = 'P_Short'
# stats_variable_name: str = 'short_best_direction_change_bin_ratio'
# stats_variable_name: str = 'short_best_wcorr'
shuffle_results, p_value, f_value, (dof1, dof2), (variance1, variance2) = _perform_stats_tests(valid_ripple_df, stats_variable_name='short_best_wcorr')


In [ ]:
# Show that wcorr in both periods is higher than shuffles


In [ ]:
## User non-selected:
scatter_title = f'user_approved_ripple_df Several Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_approved_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples

In [ ]:
# IDEA: The ones with clear replays (diagonal sequences in the decoded posteriors) are by definiition ambiguous, because there's not much difference between the long/short decoders.


In [ ]:
## User non-selected:
scatter_title = f'Non-selected Several Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_rejected_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


In [ ]:
# Laps test
concatenated_ripple_df = deepcopy(all_sessions_simple_pearson_laps_df)

scatter_title = 'Several Sessions'
variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} 
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


# Plotting functions

In [ ]:
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import PlottingHelpers
# from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import plot_across_sessions_scatter_results

# Example usage:
all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode)
fig_laps, fig_ripples = all_session_figures[0]
# fig_laps.update_layout(fig_size_kwargs)
# fig_ripples.update_layout(fig_size_kwargs)

# fig_laps.show()
fig_ripples.show()
# fig_laps.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_laps.html")
# fig_ripples.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_ripples.html")


In [ ]:
figure_context = figure_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)



In [ ]:
fig_to_clipboard(fig_ripples)

In [ ]:
## time_bin version:
all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='separate_row_per_session',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode)
fig_time_bin_laps, fig_time_bin_ripples = all_time_bin_session_figures[0]
# fig_time_bin_laps.show()
fig_time_bin_ripples.show()

In [ ]:
## Test collapsed histograms-only results:
histograms_only_all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
															# enabled_time_bin_sizes=[0.03, 0.058, 0.10], # [0.03 , 0.044, 0.058, 0.072, 0.086, 0.1]
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='default',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=False, figure_save_extension=['.html','.png'])
histograms_only_fig_time_bin_laps, histograms_only_fig_time_bin_ripples = histograms_only_all_time_bin_session_figures[0]
# histograms_only_fig_time_bin_laps.show()
histograms_only_fig_time_bin_ripples.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go



all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=single_time_bin_size_all_sessions_laps_df, concatenated_ripple_df=single_time_bin_size_all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode, main_plot_mode='default')
fig_laps, fig_ripples = all_session_figures[0]


In [ ]:
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_pre_post_delta_scatter

# data_results_df: pd.DataFrame = deepcopy(all_sessions_laps_df) # all_sessions_laps_time_bin_df
data_results_df: pd.DataFrame = deepcopy(single_time_bin_size_all_sessions_laps_time_bin_df).infer_objects()

# histogram_bins = 'auto'
histogram_bins: int = 25
new_laps_fig, new_laps_fig_context = plotly_pre_post_delta_scatter(data_results_df=data_results_df, out_scatter_fig=fig_laps, histogram_bins=histogram_bins, px_scatter_kwargs=dict(title='Laps'))


# new_laps_fig = new_laps_fig.update_layout(fig_size_kwargs, 
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     ),
# )
# Update x-axis labels
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=1)
# new_laps_fig.update_xaxes(title_text="Delta-aligned Event Time (seconds)", row=1, col=2)
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=3)


_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_laps_fig.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')

new_laps_fig

# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)

new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='laps', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_laps_fig, a_fig_context=new_laps_fig_context)
new_laps_fig

In [ ]:
data_results_df


In [ ]:
np.unique(data_results_df['time_bin_size'])


In [ ]:

data_results_df[data_results_df['time_bin_size'] == 0.005]

# Matplotlib-based versions:

In [ ]:
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_stacked_histograms

matplotlib.use('Qt5Agg')

# #TODO 2024-06-05 09:15: - [ ] Define save_matplotlib function
new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='laps', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)

In [ ]:
# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:'long_best_dir_simple_pearsonr'
_out0: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Laps', session_spec='All Sessions', data_results_df=all_sessions_laps_time_bin_df, time_bin_duration_str="75 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Ripples', session_spec='All Sessions', data_results_df=all_sessions_ripple_time_bin_df, time_bin_duration_str="75 ms")

In [ ]:


fig_to_clipboard(_out0.figures[0])

In [ ]:
## all_sessions_new_laps_df, all_sessions_new_ripple_df
_out0: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='New Laps', session_spec='All Sessions', data_results_df=all_sessions_new_laps_df, time_bin_duration_str="25 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='New Ripples', session_spec='All Sessions', data_results_df=all_sessions_new_ripple_df, time_bin_duration_str="25 ms")



In [ ]:
all_sessions_new_laps_df

In [ ]:
len(all_sessions_new_laps_df['session_name'].unique()) # 10 sessions



In [ ]:

matplotlib.use('Qt5Agg')

column_name: str = 'P_Short'

# You can use it like this:
num_unique_sessions: int = all_sessions_laps_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_laps_time_bin_df.time_bin_size.nunique(dropna=True)
_laps_histogram_out = plot_stacked_histograms(all_sessions_laps_time_bin_df, 'Laps', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes", column_name=column_name)

fig_to_clipboard(_laps_histogram_out.figures[0])

num_unique_sessions: int = all_sessions_ripple_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_ripple_time_bin_df.time_bin_size.nunique(dropna=True)
_ripple_histogram_out = plot_stacked_histograms(all_sessions_ripple_time_bin_df, 'Ripples', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes", column_name=column_name)



In [ ]:
all_sessions_ripple_time_bin_df['time_bin_size'].unique()

In [ ]:
# active_time_bin_size: float = 0.072 # 0.058
# active_time_bin_size: float = 0.086
# active_time_bin_size: float = 0.1 # looks the most bimodal with as little of an intermediate value as possible.
active_time_bin_size: float = 0.13444444 # looks the most bimodal with as little of an intermediate value as possible.
_single_time_bin_ripple_histogram_out = plot_stacked_histograms(all_sessions_ripple_time_bin_df[np.isclose(all_sessions_ripple_time_bin_df['time_bin_size'], active_time_bin_size)], 'Ripples', f'{num_unique_sessions} Sessions', f"tbin: {active_time_bin_size:0.3f}s", column_name=column_name)
fig_to_clipboard(_single_time_bin_ripple_histogram_out.figures[0], bbox_inches='tight')


In [ ]:
fig_to_clipboard(_ripple_histogram_out.figures[0])

In [ ]:
_laps_histogram_out.figures[0]

figures_folder

figures_folder

# 2024-03-28 - AcrossSessionTable (PhoDibaPaper2023 formats) .h5 and figure exports

In [25]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots


neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True)
num_sessions: int = len(long_short_fr_indicies_analysis_table['session_uid'].unique())
print(f'num_sessions: {num_sessions}')

long_short_fr_indicies_analysis_table


def _save_matplotlib_fig(matplotlib_output_container: MatplotlibRenderPlots):

    a_fig_context = matplotlib_output_container.context
    assert len(matplotlib_output_container.saved_figures) == 1
    a_saved_fig_path = matplotlib_output_container.saved_figures[0][0]
    assert a_saved_fig_path.exists()

    if neptuner_run is not None:
        a_full_figure_path_key: str = a_fig_context.get_description(separator='/', include_property_names=True, key_value_separator=':') # .replace(' ', '_')
        # a_full_figure_path_key: str = a_fig_context.get_description(separator=':', include_property_names=True, key_value_separator='|')
        print(f'a_full_figure_path_key: "{a_full_figure_path_key}"')
        # neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_fig)
        neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_saved_fig_path.as_posix())
        # neptuner.figures[f"{a_full_figure_path_key}"].upload(a_fig)
        


failed for file path: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee_2006-6-12_16-53-46_pipeline_results.h5, table_key: /kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/two/2006-6-12_16-53-46/neuron_identities/table``. Skipping.
failed for file path: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee_11-02_17-46-44_pipeline_results.h5, table_key: /kdiba/pin01/one/11-02_17-46-44/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-02_17-46-44/neuron_identities/table``. Skipping.
concatenating dataframes from 12 of 14 files
failed for file path: /Users/pho/University of Michigan Dropbox/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs/2024-06-12_Apogee_2006-6-12_16-53-46_pipeline_results.h5, table_key: /kdib

In [24]:
# matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots

matplotlib.use('Qt5Agg')
matplotlib_output_container: MatplotlibRenderPlots = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions)

/var/folders/c7/74vbnwdj7531qy2glkcjq6jr0000gn/T/ipykernel_55388/2123512628.py:4: MatplotlibDeprecationWarning:

Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.



register_output_file(output_path: /Users/pho/repo/Pho Secondary Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/SCRATCH/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2024-06-13/kdiba/{}_13_across_sessions_firing_rate_index.png, ...)
	 saved "file:///Users/pho/repo/Pho%20Secondary%20Workspace/Spike3DEnv/Spike3DWorkEnv/Spike3D/SCRATCH/EXTERNAL/Screenshots/ProgrammaticDisplayFunctionTesting/2024-06-13/kdiba/%7B%7D_13_across_sessions_firing_rate_index.png"


In [ ]:
# graphics_output_dict.figures[0]
# graphics_output_dict.context
_save_matplotlib_fig(matplotlib_output_container)

In [ ]:
graphics_output_dict.saved_figures[0][0] #.plot_data['saved_figures']

In [ ]:
copy_image_to_clipboard(graphics_output_dict['figures'][0])

In [ ]:
neptuner.stop()